<a href="https://colab.research.google.com/github/nobububi/Brandear/blob/master/code/03_01_%E5%9F%BA%E6%9C%AC%E3%83%86%E3%82%99%E3%83%BC%E3%82%BF%E7%B5%90%E5%90%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import glob
import os

import pandas as pd
from google.colab import drive

In [2]:
DATA_DIR = "drive/MyDrive/data/"

INPUT_DIR = f"{DATA_DIR}baoz/unzip/DB/"
INPUT_shussoubamasuta_FILE = f"{DATA_DIR}tiho_work/preprocess/00_01_se_mdb_shussoubamasuta.pickle"

OUTPUT_FILE = f"{DATA_DIR}tiho_work/preprocess/03_01_race_jisseki_df.pickle"

In [3]:
# 自分のドライブ配下をマウント
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
print(datetime.datetime.now())

2022-07-16 07:32:39.813444


## 各データ読み込み

### BaoZ-SE.mdb_shussoubamasuta

In [5]:
# baoz_se_shussoubamasuta_df = pd.read_csv(f"{INPUT_DIR}BaoZ-SE.mdb_shussoubamasuta.csv")
baoz_se_shussoubamasuta_df = pd.read_pickle(INPUT_shussoubamasuta_FILE)

In [6]:
# 2005年以降のデータに絞る
baoz_se_shussoubamasuta_df["年"] = baoz_se_shussoubamasuta_df["開催年月日"].str[:4]
baoz_se_shussoubamasuta_df["年"].value_counts().sort_index()

1954         4
1955        10
1956         7
1959         3
1960         8
1961         2
1962         4
1963         3
1964         8
1965         8
1967         8
1968        10
1969         9
1975         3
1976        21
1977        16
1978         1
1982         3
1983         9
1984        11
1985        34
1986     36385
1987     36751
1988     37439
1989     38024
1990     38780
1991     41215
1992     45863
1993     51916
1994     55477
1995     59658
1996     62906
1997     71985
1998     80215
1999     84192
2000     71977
2001     77287
2002     85692
2003     92486
2004     88283
2005    209758
2006    203213
2007    186372
2008    189251
2009    192615
2010    190691
2011    183466
2012    184812
2013    180664
2014    177421
2015    176702
2016    178408
2017    179538
2018    179939
2019    182342
2020    183791
2021    179679
2022     82939
Name: 年, dtype: int64

In [7]:
baoz_se_shussoubamasuta_df = baoz_se_shussoubamasuta_df[baoz_se_shussoubamasuta_df["年"].astype(int) >= 2005].reset_index(drop=True)

In [8]:
baoz_se_shussoubamasuta_df["競走コード"] = baoz_se_shussoubamasuta_df["競走コード"].astype(int)

### BaoZ.ex.mdb_shussoubaT

In [9]:
baoz_ex_shussouba_df = pd.read_csv(f"{INPUT_DIR}BaoZ.ex.mdb_shussoubaT.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (0,86) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [10]:
baoz_ex_shussouba_df["競走コード"] = baoz_ex_shussouba_df["競走コード"].astype(int)

In [11]:
# 2010年以前はデータ件数が少ない
baoz_ex_shussouba_df["年"] = baoz_ex_shussouba_df["年月日"].str[:4]
baoz_ex_shussouba_df["年"].value_counts().sort_index()

2000     71629
2001     77287
2002     85692
2003     92486
2004     88283
2005     48758
2006     49785
2007     49447
2008     50909
2009     50927
2010    190691
2011    183466
2012    184812
2013    180656
2014    177421
2015    176702
2016    178408
2017    179538
2018    179939
2019    182342
2020    183791
2021    179679
2022     82939
Name: 年, dtype: int64

### BaoZ-RA.mdb_reesumasuta

In [12]:
baoz_ra_reesumasuta_df = pd.read_csv(f"{INPUT_DIR}BaoZ-RA.mdb_reesumasuta.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (14,15,16,23,79) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [13]:
baoz_ra_reesumasuta_df.shape

(399691, 80)

In [14]:
baoz_ra_reesumasuta_df["競走コード"] = baoz_ra_reesumasuta_df["競走コード"].astype(int)

### BaoZ-RA.mdb_kyousoubamasuta.csv

In [15]:
baoz_ra_kyousoubamasuta_df = pd.read_csv(f"{INPUT_DIR}BaoZ-RA.mdb_kyousoubamasuta.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (46) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [16]:
baoz_ra_kyousoubamasuta_df.shape

(230547, 87)

## 結合

In [17]:
print(baoz_se_shussoubamasuta_df.shape)
print(baoz_ex_shussouba_df.shape)

(3241601, 65)
(2925587, 113)


In [18]:
baoz_ex_shussouba_df["baoz_ex_shussouba_df_flag"] = 1
baoz_ra_reesumasuta_df["baoz_ra_reesumasuta_df_flag"] = 1
baoz_ra_kyousoubamasuta_df["baoz_ra_kyousoubamasuta_df"] = 1

In [19]:
print(datetime.datetime.now())

# 3min
# TODO: 紐付き件数チェックしておく、baoz_ex_shussouba_dfがあんまり紐づいてない気がする

merge_dupcol_suf = "_merge_dupcol"

race_jisseki_df = (
    baoz_se_shussoubamasuta_df
    .merge(baoz_ex_shussouba_df, on=["競走コード", "馬番"], how="left", suffixes=['', merge_dupcol_suf])
    # 競走コード213011455180205が紐づかない
    .merge(baoz_ra_reesumasuta_df, on=["競走コード"], how="inner", suffixes=['', merge_dupcol_suf])
    .merge(baoz_ra_kyousoubamasuta_df, on=["血統登録番号"], how="left", suffixes=['', merge_dupcol_suf])
)

print(datetime.datetime.now())

2022-07-16 07:34:47.269029
2022-07-16 07:37:38.570076


### 紐付き確認

In [20]:
print(baoz_se_shussoubamasuta_df.shape)

(3241601, 65)


In [21]:
print(race_jisseki_df.shape)
join_check_cols = [
    "baoz_ex_shussouba_df_flag",
    "baoz_ra_reesumasuta_df_flag",
    "baoz_ra_kyousoubamasuta_df"
]
for check_col in join_check_cols:
    print(check_col)
    print(race_jisseki_df[check_col].isnull().sum())

(3241593, 344)
baoz_ex_shussouba_df_flag
731383
baoz_ra_reesumasuta_df_flag
0
baoz_ra_kyousoubamasuta_df
25927


In [22]:
race_jisseki_df.drop([c for c in race_jisseki_df.columns if c.endswith(merge_dupcol_suf)], axis=1, inplace=True)

## 出力

In [23]:
race_jisseki_df.to_pickle(OUTPUT_FILE)

In [24]:
print(datetime.datetime.now())

2022-07-16 07:40:24.713349
